## Importing libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Reshape
#from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.utils import load_img,img_to_array
import glob
from tqdm import tqdm
import warnings;
warnings.filterwarnings('ignore')
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD

## Reading the resized images

In [2]:
file = open('img_x_train.pkl','rb')

resized_img_Xtrain = pickle.load(file)

file.close()
file = open('img_y_train.pkl','rb')

resized_img_Ytrain = pickle.load(file)

file.close()
file = open('img_x_test.pkl','rb')

resized_img_Xtest = pickle.load(file)

file.close()
file = open('img_y_test.pkl','rb')

resized_img_Ytest = pickle.load(file)

file.close()
file = open('img_x_val.pkl','rb')

resized_img_Xval = pickle.load(file)

file.close()
file = open('img_y_val.pkl','rb')

resized_img_Yval = pickle.load(file)

file.close()

## Normalizing the images

In [ ]:
train_x = np.array(resized_img_Ytrain)
train_x = train_x/255.0
test_x = np.array(resized_img_Ytest)
test_x = test_x/255.0
val_x = np.array(resized_img_Yval)
val_x = val_x/255.0
train_x_px = np.array(resized_img_Xtrain)
train_x_px = train_x_px/255.0
test_x_px = np.array(resized_img_Xtest)
test_x_px = test_x_px/255.0
val_x_px = np.array(resized_img_Xval)
val_x_px = val_x_px/255.0

## Autoencoder - Fully connected 1 Low risk

In [ ]:
encoder = Sequential()

encoder.add(Flatten(input_shape=[80,80,3]))
encoder.add(Dense(9600,activation="relu"))
encoder.add(Dense(4800,activation="relu"))
encoder.add(Dense(2400,activation="relu"))
encoder.add(Dense(1200,activation="relu"))
encoder.add(Dense(600,activation="relu"))


decoder = Sequential()

decoder.add(Dense(1200,input_shape=[600],activation='relu'))
decoder.add(Dense(2400,activation='relu'))
decoder.add(Dense(4800,activation='relu'))
decoder.add(Dense(9600,activation='relu'))
decoder.add(Dense(80*80*3,activation='sigmoid',input_shape = [9600]))
decoder.add(Reshape([80,80,3]))

full_autoencoder = Sequential([encoder,decoder])

full_autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

full_autoencoder.fit(train_x_px, train_x,
            epochs=50,
            batch_size=32,
            shuffle=True,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper]
            )

In [ ]:
predictions = simple_autoencoder.predict(test_x_px)

In [ ]:
n = 5
plt.figure(figsize= (20,10))

for i in range(n):
  ax = plt.subplot(2, n, i+1)
  plt.imshow(test_x_px[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(predictions[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

## Autoencoder - Fully connected 2 Low risk

In [ ]:
encoder = Sequential()

encoder.add(Flatten(input_shape=[80,80,3]))
encoder.add(Dense(9600,activation="relu"))



decoder = Sequential()

decoder.add(Dense(80*80*3,input_shape=[9600],activation='sigmoid'))

decoder.add(Reshape([80,80,3]))

simple_autoencoder = Sequential([encoder,decoder])



In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

simple_autoencoder.fit(train_x_px, train_x,
            epochs=50,
            batch_size=32,
            shuffle=True,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper]
            )

In [ ]:
predictions = simple_autoencoder.predict(test_x_px)

In [ ]:
n = 5
plt.figure(figsize= (20,10))

for i in range(n):
  ax = plt.subplot(2, n, i+1)
  plt.imshow(test_x_px[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(predictions[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
simple_autoencoder.save('simple_autoencoder.h5')

### Convulation encoder High risk

In [ ]:
Input_img = Input(shape=(80, 80, 3))  
   
#encoding architecture
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x2 = MaxPool2D( (2, 2))(x2)
encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

# decoding architecture
x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x3 = UpSampling2D((2, 2))(x3)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
decoded = Conv2D(3, (3, 3), padding='same')(x1)

autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

autoencoder.fit(train_x_px, train_x,
            epochs=50,
            batch_size=32,
            shuffle=True,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper]
            )